# First-class Module

##### 🟢 &nbsp; &nbsp; Packing a module as a first-class value and unpacking it into a module.

In [12]:
module type T = sig val x : int end
module M  = struct let x = 25 end

(* packing module to a value *)
let packed = (module M : T) ;;

(* unpacking module value *)
let module Unpacked = (val packed : T ) in
Unpacked.x;;

(* unpacking module value using pattern match *)
let (module Unpacked : T ) = packed in
Unpacked.x ;;

module type T = sig val x : int end


module M : sig val x : int end


val packed : (module T) = <module>


- : int = 25


- : int = 25


<br/>

##### 🟢 &nbsp; &nbsp; Passing a packed module value as a function paramater

In [13]:
module type T = sig val x : int end
module M  = struct let x = 25 end
let packed = (module M : T) ;;
let f m = 
    let module Unpacked = (val m : T) in
    Unpacked.x ;;
f packed ;;

(* using pattern match *)
let f (module M : T) = M.x ;;
f packed ;;

module type T = sig val x : int end


module M : sig val x : int end


val packed : (module T) = <module>


val f : (module T) -> int = <fun>


- : int = 25


val f : (module T) -> int = <fun>


- : int = 25


<br/>

##### 🟢 &nbsp; &nbsp; Returning a packed module value from a function

In [14]:
module type T = sig val x : int end ;;
let foo (v: int) = (module struct let x = v end : T) in
let (module M : T) = foo 100 in
M.x ;;

module type T = sig val x : int end


- : int = 100


<br/>

##### 🟢 &nbsp; &nbsp; With module type having abstract type

In [15]:
module type T = sig 
    type t
    val double : t -> t
end 

module M = struct
    type t = int
    let double x = 2 * x
end ;;

let packed = (module M : T with type t = int) in
let module Unpacked = (val packed: T with type t = int) in
Unpacked.double 2 ;;

module type T = sig type t val double : t -> t end


module M : sig type t = int val double : int -> int end


- : int = 4


<br/>

##### 🟢 &nbsp; &nbsp; With locally abstract type

In [16]:
module type T = sig 
    type t
    val double : t -> t
end 

module M_Int = struct
    type t = int
    let double x = 2 * x
end

module M_Float = struct
    type t = float
    let double x = 2. *. x
end

let double_list (type a) (module M : T with type t = a) lst = 
    lst |> List.map M.double ;;


double_list (module M_Int) [1;2;3] ;;

double_list (module M_Float) [1.;2.;3.] ;;

module type T = sig type t val double : t -> t end


module M_Int : sig type t = int val double : int -> int end


module M_Float : sig type t = float val double : float -> float end


val double_list : (module T with type t = 'a) -> 'a list -> 'a list = <fun>


- : M_Int.t list = [2; 4; 6]


- : M_Float.t list = [2.; 4.; 6.]


<br/>

##### 🟢 &nbsp; &nbsp; Sample

In [17]:
module type SHAPE = sig    
    val draw : unit -> string
end

module Circle  = struct
    let location = (1., 2.)
    let draw () = Printf.sprintf ("Drawing a circle at (%F, %F).") (fst location) (snd location)
end

module Rectangle  = struct
    let width = 100.
    let height = 50.
    let draw () = Printf.sprintf ("Drawing a rectangle with width = %F, height = %F.") width height
end ;;

let draw (module S : SHAPE) = S.draw() ;; 
draw (module Circle : SHAPE) ;;
draw (module Rectangle : SHAPE) ;;

let shapes = [ (module Circle : SHAPE) ; (module Rectangle); (module struct let draw () = "Anonymous Shape" end) ] ;;
let (module MP : SHAPE) = List.hd shapes in
MP.draw() ;;

let rec display (lst : (module SHAPE) list) ret : string = 
    match lst with
    | [] -> ret
    | h :: t -> 
        let (module MS : SHAPE) = h in
        display t (ret ^ MS.draw()) ;;
display shapes "" ;;

module type SHAPE = sig val draw : unit -> string end


module Circle :
  sig val location : float * float val draw : unit -> string end


module Rectangle :
  sig val width : float val height : float val draw : unit -> string end


val draw : (module SHAPE) -> string = <fun>


- : string = "Drawing a circle at (1., 2.)."


- : string = "Drawing a rectangle with width = 100., height = 50.."


val shapes : (module SHAPE) list = [<module>; <module>; <module>]


- : string = "Drawing a circle at (1., 2.)."


val display : (module SHAPE) list -> string -> string = <fun>


- : string =
"Drawing a circle at (1., 2.).Drawing a rectangle with width = 100., height = 50..Anonymous Shape"


<br/>

##### 🟢 &nbsp; &nbsp; Sample from Manual https://v2.ocaml.org/manual/firstclassmodules.html 

In [18]:
let make_set (type a) cmp =
    let module S = Set.Make(struct
        type t = a
        let compare = cmp
    end) in
    (module S : Set.S with type elt = a) ;;
    
let sort (type a) (module S : Set.S with type elt = a) l =
    S.elements (List.fold_right S.add l S.empty) ;;

let s = make_set Int.compare in
sort s [0; 3; 1; 4; 3; 2; 1; 100; 9] ;;

let s = make_set Float.compare in
sort s [0.; 3.; 1.; 4.; 3.; 2.; 1.; 100.; 9.] ;;

val make_set : ('a -> 'a -> int) -> (module Set.S with type elt = 'a) = <fun>


val sort : (module Set.S with type elt = 'a) -> 'a list -> 'a list = <fun>


- : int list = [0; 1; 2; 3; 4; 9; 100]


- : Float.t list = [0.; 1.; 2.; 3.; 4.; 9.; 100.]


<br/>

##### 🟢 &nbsp; &nbsp; Samples

In [19]:
let countBy (type k) (lst: 'a list) ~compare:cmp ~(projection: 'a -> k) =
    let len = List.length lst in
    if len = 0 then []
    else
        let module M = Map.Make(struct
            type t = k
            let compare = cmp
        end) in
        let rec loop l acc =
            match l with
            | [] -> acc
            | h :: t -> 
                let key = projection h in
                loop t (acc |> M.update key (function None -> Some (1) | Some x -> Some (x + 1)))
        in
        loop lst M.empty |> M.bindings ;;

let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> countBy ~compare:Int.compare ~projection:(fun x -> x) ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> countBy ~compare:Float.compare ~projection:(fun x -> x) ;;

type person = { name : string; age : float } ;;
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] in
people |> countBy ~compare:String.compare ~projection:(fun x -> x.name) ;;

val countBy :
  'a list ->
  compare:('k -> 'k -> int) -> projection:('a -> 'k) -> ('k * int) list =
  <fun>


- : (int * int) list = [(1, 2); (2, 3); (3, 2); (4, 2)]


- : (Float.t * int) list = [(1., 2); (2., 3); (3., 2); (4., 2)]


type person = { name : string; age : float; }


- : (String.t * int) list = [("Click", 3); ("Jack", 2)]


In [20]:
let distinct (type a) lst ~compare:cmp =
    let module S = Set.Make(struct 
        type t = a
        let compare = cmp
    end) in
    S.of_list lst |> S.elements ;;
    
let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> distinct ~compare:Int.compare ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> distinct ~compare:Float.compare ;;

type person = { name : string; age : float } ;;
let pcmp x y = 
    match (String.compare (String.lowercase_ascii x.name) (String.lowercase_ascii y.name), Float.compare x.age y.age) with
    | (0, 0) -> 0
    | (a, b) -> if a > b then a else b ;;
    
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] in
people |> distinct ~compare:pcmp ;;

val distinct : 'a list -> compare:('a -> 'a -> int) -> 'a list = <fun>


- : int list = [1; 2; 3; 4]


- : Float.t list = [1.; 2.; 3.; 4.]


type person = { name : string; age : float; }


val pcmp : person -> person -> int = <fun>


- : person list =
[{name = "Click"; age = 13.}; {name = "Jack"; age = 13.};
 {name = "Jack"; age = 14.}; {name = "Click"; age = 31.}]


In [21]:
let distinctBy (type k) (lst:'a list) ~(compare:k -> k -> int) ~(projection:'a -> k) =
    let module S = Set.Make(struct 
        type t = k
        let compare = compare
    end) in
    let rec loop l keys ret =
        match l with
        | [] -> ret
        | h :: t -> 
            let key = projection h in
            if keys |> S.exists (fun x -> x = key) then loop t keys ret
            else loop t (keys |> S.add key) (h :: ret)               
    in
    loop lst S.empty [] |> List.rev ;;

let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> distinctBy ~compare:Int.compare ~projection:(fun x -> x) ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> distinctBy ~compare:Float.compare ~projection:(fun x -> x) ;;

type person = { name : string; age : float } ;;
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] in
people |> distinctBy ~compare:String.compare ~projection:(fun x -> x.name) ;;


val distinctBy :
  'a list -> compare:('k -> 'k -> int) -> projection:('a -> 'k) -> 'a list =
  <fun>


- : int list = [1; 2; 3; 4]


- : Float.t list = [1.; 2.; 3.; 4.]


type person = { name : string; age : float; }


- : person list = [{name = "Click"; age = 13.}; {name = "Jack"; age = 13.}]


In [22]:
let groupBy (type k) (lst:'a list) ~(compare:k -> k -> int) ~(projection: 'a -> k) =
    let module M = Map.Make(struct 
        type t = k
        let compare = compare
    end) in
    let rec loop l ret =
        match l with
        | [] -> ret
        | h :: t ->
            let key = projection h in
            loop t (ret |> M.update key (function  Some x -> Some ( h :: x ) | _ -> Some [h]))
    in
    loop lst M.empty |> M.bindings ;;
    
let ints = [1; 1; 2; 2; 2; 3; 3; 4; 4] in
ints |> groupBy ~compare:Int.compare ~projection:(fun x -> x) ;;

let floats = [1.; 1.; 2.; 2.; 2.; 3.; 3.; 4.; 4.] in
floats |> groupBy ~compare:Float.compare ~projection:(fun x -> x) ;;

type person = { name : string; age : float } ;;
let people = [ 
    { name = "Click"; age = 13. };
    { name = "Click"; age = 12. }; 
    { name = "Jack"; age = 13. };
    { name = "Jack"; age = 14. };
    { name = "Click"; age = 31. }
] in
people |> groupBy ~compare:String.compare ~projection:(fun x -> x.name) ;;

val groupBy :
  'a list ->
  compare:('k -> 'k -> int) -> projection:('a -> 'k) -> ('k * 'a list) list =
  <fun>


- : (int * int list) list =
[(1, [1; 1]); (2, [2; 2; 2]); (3, [3; 3]); (4, [4; 4])]


- : (Float.t * Float.t list) list =
[(1., [1.; 1.]); (2., [2.; 2.; 2.]); (3., [3.; 3.]); (4., [4.; 4.])]


type person = { name : string; age : float; }


- : (String.t * person list) list =
[("Click",
  [{name = "Click"; age = 31.}; {name = "Click"; age = 12.};
   {name = "Click"; age = 13.}]);
 ("Jack", [{name = "Jack"; age = 14.}; {name = "Jack"; age = 13.}])]
